# Run: rdrp1 Error Sweep (QC)

```
Lead     : ababaian
Issue    : 
Version  : v0.4.0-dev : diamond-dev branch
start    : 2021 01 16
complete : 2021 01 xx
files    : ~/serratus/notebook/210108_ab/
s3_files : s3://serratus-public/notebook/210108_ab/
output   : s3://serratus-public/out/210118_rQC/
```

### Intro/Objectives

> Heart is pounding in his chest, Reaches suddenly for his gun, Should be sailing in the West, Flower that could be his fun

### QC

- File: `qc_SraRunInfo.csv`

Results: `132 165 / 160 372`

[See: 201230_sraRunInfo_Update.ipynb](./201230_sraRunInfo_Update.ipynb)


In [ ]:
# ON EC2
# Make QC SRA list
mkdir -p sra; cd sra
mkdir -p qc

aws s3 cp s3://serratus-public/tmp/completed.sra ./
aws s3 sync s3://serratus-public/notebook/201230_sra/ ./

# do not inlude prokaryotes
rm pro*
cat *.csv > qc/global_SraRunInfo.csv

remove_done () {
  COMPLETE=$1
  SRA=$2
  
  # Sort unique SRA in the input RunInfo file
  cut -f1 -d',' $SRA \
    | sort -u \
    > inputsra.tmp
  
  # Ensure completed SRA list is sorted
  sort $COMPLETE > complete.sort.tmp
  
  # Supress SRA-accessions only in file 1 or file 1+2
  # Show SRA unique to file 2
  # parse into a regex expression to find matching lines
  comm -1 -3 complete.sort.tmp inputsra.tmp \
    | sed 's/^$\n//g' - \
    | sed 's/^/^/g' - \
    | sed 's/$/,/g' \
    > incomplete.in.$SRA
  
  split -l 10000 incomplete.in.$SRA inc.split.
  
  rm -f undone_$SRA
  for INC in $(ls inc.split.*); do
    grep -f $INC $SRA >> undone_$SRA
  done
  
  
  #grep -f incomplete.in.$SRA $SRA > undone_$SRA
  
  #rm *.tmp
  # Still crashing due to high memory
  # but will give a list of "undone" accessions
}

remove_done completed.sra global_SraRunInfo.csv

head -n1 global_SraRunInfo.csv > qc_SraRunInfo.csv
sort -u undone_global_SraRunInfo.csv >> qc_SraRunInfo.csv

# Manually fix header column (remove duplicates)
aws s3 cp qc_SraRunInfo.csv s3://serratus-public/notebook/201230_sra/

In [ ]:
## Fire up EC2 Instance
#sudo yum install -y docker
#sudo yum install -y git
#sudo service docker start

## Download latest serratus repo
#git clone -b diamond-dev https://github.com/ababaian/serratus.git; cd serratus/containers

## If you want to upload containers to your repository, include this.
#export DOCKERHUB_USER='serratusbio' # optional
#sudo docker login # optional
#
## Build all containers and upload them docker hub repo (if available)
#./build_containers.sh

### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/210111_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/210111_ab'

# date and version
date
git rev-parse HEAD # commit version

Mon Jan 18 16:40:37 PST 2021
2ddf24166fdded0bc89a96b2541bcbc7442f98f9


In [2]:
cd $WORK

# Copy SRA Run Info
aws s3 cp s3://serratus-public/notebook/201230_sra/qc_SraRunInfo.csv ./

md5sum qc*
ls -alh qc*

download: s3://serratus-public/notebook/201230_sra/qc_SraRunInfo.csv to ./qc_SraRunInfo.csv


7a413537e9e48306b495d446336b0374  qc_SraRunInfo.csv
-rw-rw-r-- 1 artem artem 70M Jan 18 16:39 qc_SraRunInfo.csv


In [ ]:
#aws s3 sync ./ $S3_WORK/

### Terraform Initialize

In [ ]:
## UPDATE serratus/terraform/main/main.tf
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

In [3]:
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..d96bf62 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,68 +89,73 @@ re

     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corres

module.download.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creation complete after 1s [id=SerratusIamRole-serratus-dl-20210119004500170300000004]
module.align.aws_iam_role_policy.ec2Terminate: Creating...
module.merge.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creation complete after 1s [id=SerratusIamRole-serratus-merge-20210119004500336600000005]
module.merge.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-merge:CloudWatchLogsCreate-serratus-merge]
module.align.aws_iam_role_policy.AdjustAutoScaling: Creating...
module.scheduler.module.ecs_cluster.aws_iam_role_policy_attachment.instance_attachment: Creating...
module.download.module.iam_role.aws_iam_instance_profile.profile: Creation complete after 1s [id=profile-serratus-dl]
aws_security_group.internal: Creation complete after 5s [id=sg-071b9a4202af

module.scheduler.aws_eip.sch: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 1s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.scheduler.aws_eip.sch: Creation complete after 3s [id=eipalloc-0ac3d53bd4056f829]
module.merge.aws_launch_configuration.worker: Creating...
module.align.aws_launch_configuration.worker: Creating...
module.download.aws_launch_configuration.worker: Creating...
module.monitoring.aws_eip.monitor: Creation complete after 4s [id=eipalloc-0336cd3ddd37710f2]
local_file.create_tunnel: Creating...
local_file.hosts: Creating...
local_file.create_tunnel: Creation complete after 0s [id=362587148693f2516a3983b42e90346fa9dc6964]
local_file.hosts: Creation complete after 0s [id=d4ff1a1817f180971ee0f

Serratus Config backup 
```
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 40,
  "ALIGN_SCALING_CONSTANT": 0.08,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 3200,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 20,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 1350,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 25,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}
```

In [4]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>

Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [9]:
cd $WORK
BATCH='qc_SraRunInfo.csv'
wc -l $WORK/$BATCH

160372 /home/artem/serratus/notebook/210111_ab/qc_SraRunInfo.csv


In [10]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/qc_SraRunInfo.csv
  date: Mon Jan 18 16:48:25 PST 2021
  wc  : 160372 /home/artem/serratus/notebook/210111_ab/qc_SraRunInfo.csv
  md5 : 7a413537e9e48306b495d446336b0374  /home/artem/serratus/notebook/210111_ab/qc_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
13da9551c317a6c85dda398675407c2a  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
c95db77c73ac15220dc8735a80068673  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
447fa71b67a593886b0c0432f1ddac7d  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
fba5819da3df646062f7ff234670ed6d  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":10000,"to

## Run Serratus

In [16]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json

cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -sT serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 50,
  "ALIGN_SCALING_CONSTANT": 0.2,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 10,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 20,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 0,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 45,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 20,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}

{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":50,"ALIGN_SCALING_CONSTANT":0.2,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":10,"CLEAR_INTERVAL":999999,"DL_ARGS":"","DL_MAX_INCREASE":20,"DL_SCALING_CONSTANT":0.1,"DL_SCALING_ENABLE":true,"DL_SCALING_MAX":0,"GENOME":"rdrp1","MERGE_ARGS":"protein","MERGE_MAX_INCREASE":45,"MERGE_SCALING_CONSTANT":0.1,"MERGE_SCALING_ENABLE":true,"MERGE_SCALING_MAX":20,"SCALING_INTER

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Regular Shutdown
cd $TF
terraform destroy

## Nuke Shutdown
#cd $TF
#
#aws ec2 describe-instances \
#  --filter Name=tag:Name,Values=serratus-align-instance \
#  > align_instances.json
#
#jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
#  | pv -l \
#  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

# Notes: 



psummary: `https://serratus-public.s3.amazonaws.com/out/210108_r1p/psummary/<SRA>.psummary`

SRA site: `https://www.ncbi.nlm.nih.gov/sra/?term=<SRA>`
